# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP2 - Recherche Adversarielle 
### Membres de l'équipe
    - Marc El Khoury 1798468
    - Ayoub El Asry 1800595




## Directives de remise
Le travail sera réalisé avec la  même équipe que pour le TP1. Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **6 novembre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barème
Partie 1: 25 points

Partie 2: 35 points

Partie 3: 15 points

Partie 4: 15 points

Partie 5: 10 points

Pour un total de 100 points possibles sur 100 points.


## Recherche Adversarielle

La recherche adversarielle est une méthode d’intelligence artificielle qui est utilisée pour prédire les actions d’un adversaire dans un jeu vidéo. Il consiste à construire un arbre avec les actions possibles suivi des réactions de l’adversaire à ces actions. Pour choisir l’action optimale, il existe plusieurs méthodes. Une très commune est la recherche minimax. Elle évalue la valeur des actions et maximise la valeur de celle du joueur et minimise la valeur de celle de l’adversaire.

### Rush Hour

Vous avez vu le jeu de puzzle, Rush Hour, dans le TP précédent. Dans le cadre de ce TP, un adversaire a été ajouté pour transformer Rush Hour en jeu adversarielle. À chaque tour, l’adversaire ajoute une roche dans la grille. Cette roche disparaît après 1 tour et empêche les voitures de se déplacer. Son but est d’obliger le joueur à faire le plus de coups possible et de l’empêcher de bouger complètement. Le joueur peut déplacer une voiture, selon les règles vues dans le tp précédent, par tour. Cette version de Rush Hour est un jeu zero-sum déterministe.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

### But

Votre but dans ce TP est de développer un algorithme minimax pour permettre à votre AI de terminer le niveau de Rush Hour **avec un minimun de mouvement**. Une version complétée des classes Rush Hour et State est fournie pour vous aider à faire le TP. Ces classes vont être réutilisées pour représenter les états à évaluer. Vous pouvez les remplacer par vos propres implémentations en Python (vous pouvez aussi réécrire le TP dans le langage de votre choix).  

## 1. Implémentation de Rush Hour (25pts)
Les classes suivantes sont complètes pour représenter les différents états d’une partie de Rush Hour normale.
Il manque la représentation des coups de l’adversaire : vous auriez à implémenter cette partie-là dans les classes touchées.

L’état du stationnement au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos` : un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente la première case occupée par la voiture);

- `c` : l’index de la voiture déplacée pour atteindre l’état courant à partir de l’état précédent;

- `d` : la direction du déplacement fait à partir de l’état précédent pour atteindre l’état courant (+1 : vers la droite ou vers le bas, -1 : vers la gauche ou vers le haut);

- `prev` : l’état précédent

- `score` : le score associé à l'état

- `rock` : la position de la roche dans la grille (ligne,colonne)

In [2]:
import numpy as np
import math
import copy
from collections import deque

class State:
    
    def __init__(self, pos):
        self.pos = np.array(pos)
        self.c = self.d = self.prev = None
        self.nb_moves = 0
        self.score = 0
        
        # TODO
        self.rock = None

    def move(self, c, d):
        s = State(self.pos)
        s.prev = self
        s.pos[c] += d
        s.c = c
        s.d = d
        s.nb_moves = self.nb_moves + 1
        s.rock = self.rock
        return s

    def put_rock(self, rock_pos):
        s = State(self.pos)
        s.prev = self.prev
        s.nb_moves = self.nb_moves
        s.c = self.c 
        s.d = self.d
        s.rock = rock_pos
        return s
            
    def estimee1(self):
        # TODO
        caseFinal = 4 # 6 cases - 2 de longueurs
        return  caseFinal - self.pos[0]

    
    def get_blocking_cars(self, rh, state):
        blocking_cars = []
        # voiture critiques previous state
        for i in range(1, rh.nbcars):
            is_car_vertical =  not rh.horiz[i]
            if is_car_vertical and rh.move_on[i]  > state.pos[0] + 1:
                # check if car of size 2 is blocking
                is_car_2_block = rh.length[i] == 2 and (state.pos[i] == 1 or state.pos[i] == 2)
                is_car_3_block = rh.length[i] == 3 and state.pos[i] <= 2
                if is_car_2_block or is_car_3_block:
                    blocking_cars.append(i)
        return blocking_cars

    def score_state(self, rh):
        previous_move_car = self.c
        previous_move_direction = self.d

        if self.prev.c == self.c and self.d == -1 * self.prev.d:
            return 100
        
        blocking_cars = self.get_blocking_cars(rh, self)
        if previous_move_car == 0:
            if previous_move_direction == 1:
                return -1
            else:
                return 15
        
        score = 0.5

        if previous_move_car in blocking_cars:
            if rh.length[previous_move_car] == 3:
                if previous_move_direction == -1:
                    score += 0.5
                else:
                    nb_blocking = 0
                    col = rh.move_on[previous_move_car]
                    start_row = self.pos[previous_move_car] + 2
                    for i in range(start_row, 6):
                        if rh.free_pos[i][col] == False:
                            nb_blocking += 1
                    if nb_blocking == 0:
                        score -= 0.5
                    else:
                        score += nb_blocking
            
            else:
                if previous_move_direction == -1:
                    nb_blocking = 0
                    free = 0
                    col = rh.move_on[previous_move_car]
                    for i in reversed(range(self.pos[previous_move_car])):
                        if rh.free_pos[i][col] == False:
                            nb_blocking += 0.5
                        else:
                            free += 1
                    
                    if nb_blocking == 0:
                        score -= 0.5*2
                    else:
                        score += nb_blocking

                else:
                    nb_blocking = 0
                    col = rh.move_on[previous_move_car]
                    for i in range(self.pos[previous_move_car] + 1, 5):
                        if rh.free_pos[i][col] == False:
                            nb_blocking += 0.5
                    if nb_blocking == 0:
                        score -= 0.5
                    else:
                        score += nb_blocking
        
        if rh.horiz[self.c] == True :
            if self.d == -1:
                if self.prev.pos[self.c] + rh.length[self.c] > self.pos[0] + 2:
                    return -1
            else:
                return 0.5

        prev_blocking_cars = self.get_blocking_cars(rh, self.prev)
        if len(blocking_cars) < len(prev_blocking_cars):
            return -10
        elif len(blocking_cars) > len(prev_blocking_cars):
            return 20
        return score

    def success(self):
        return self.pos[0] == 4
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented

        
        return np.array_equal(self.pos, other.pos)
    
    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37*h + self.pos[i]
        return int(h)
    
    def __lt__(self, other):
        return (self.score) < (other.score)

La représentation du problème est faite par la classe **Rushhour** qui contient les champs suivants :

- `nbcars` : le nombre de voitures dans la grille;
- `horiz` : un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on` : un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color` : un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos` : une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouvent dans le même index dans chacun des vecteurs.

La fonction *`print_pretty_grid()`* sert à imprimer la grille en utilisant la première lettre des couleurs pour vous aider à mieux visualiser le problème.

In [3]:
class Rushhour:
    
    def __init__(self, horiz, length, move_on, color=None):
        self.nbcars = len(horiz)
        self.horiz = horiz
        self.length = length
        self.move_on = move_on
        self.color = color
        
        self.free_pos = None
    
    def init_positions(self, state):
        self.free_pos = np.ones((6, 6), dtype=bool)
        for i in range(self.nbcars):
            if self.horiz[i]:
                self.free_pos[self.move_on[i], state.pos[i]:state.pos[i]+self.length[i]] = False
            else:
                self.free_pos[state.pos[i]:state.pos[i]+self.length[i], self.move_on[i]] = False
        # TODO
        if state.rock != None:
            self.free_pos[state.rock] = False
            
    
    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(self.nbcars):
            if self.horiz[i]:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[self.move_on[i], state.pos[i]+self.length[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[self.move_on[i], state.pos[i] - 1]:
                    new_states.append(state.move(i, -1))
            else:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[state.pos[i]+self.length[i], self.move_on[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[state.pos[i] - 1, self.move_on[i]]:
                    new_states.append(state.move(i, -1))
        return new_states
    
    def possible_rock_moves(self, state):
        self.init_positions(state)
        new_states =[]
        #TODO
        for i in range(len(self.free_pos)):
            if i != 2:
                for j in range(len(self.free_pos[0])):
                    if self.free_pos[i,j] == True and (state.rock == None  or (i != state.rock[0] and j != state.rock[1])):
                        new_states.append(state.put_rock((i,j)))
        return new_states
    

    def print_pretty_grid(self, state):
        self.init_positions(state)
        grid= np.chararray((6, 6))
        grid[:]='-'
        for car in range(self.nbcars):
            for pos in range(state.pos[car], state.pos[car] +self.length[car]):
                if self.horiz[car]:
                    grid[self.move_on[car]][pos] = self.color[car][0]
                else:
                    grid[pos][self.move_on[car]] = self.color[car][0]
        if state.rock != None:
            grid[state.rock] = 'x'
        print(grid)

### 1.1 Implémentation

1. Modifier la classe State pour qu'elle contienne la position de la roche ainsi que la fonction *`put_rock()`* pour ajouter une nouvelle roche et enlever l'ancienne. Les roches sont représentées par un tuple (ligne,colonne). La fonction retourne un nouvel objet State.

2. Modifier la fonction *`init_position()`* de la classe Rushhour pour qu'elle tienne compte de l'emplacement des roches.

3. Implémenter la fonction *`possible_rock_moves()`* qui permet de trouver tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant dans la classe Rushhour. L'adversaire ne peut ni mettre de roche sur la ligne 2 ni mettre deux roches consécutives sur la même ligne ou colonne.

Utiliser la fonction *testRocks()* pour vérifier que vous avez bien pris en compte les roches. Vous devriez avoir ces résultats:

```
État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'x' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False False False False False]
 [False False False False False False]
 [False False False False  True False]]

```

Utiliser la fonction *testPossibleRockMoves()* pour vérifier que vous trouvez bien tous les coups possibles pour l'adversaire. Vous devriez trouver 7 et 3 mouvements possibles.

In [4]:
def testRocks():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s0 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    
    s1= s0.put_rock((4,4))
    s2 = s1.put_rock((3,2)) 
    
    print("État initial")
    rh.print_pretty_grid(s0)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 4-4")
    rh.print_pretty_grid(s1)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 3-2")
    rh.print_pretty_grid(s2)
    print(rh.free_pos)
    print('\n')

testRocks()

État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'

In [5]:
def testPossibleRockMoves():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    sols = rh.possible_rock_moves(s)
    print(len(sols))
    s1 = s.put_rock((3,4))
    sols = rh.possible_rock_moves(s1)
    print(len(sols))

testPossibleRockMoves()
  

7
3


## 2. Implémentation d'une recherche minimax (35pts)
L'algorithme minimax est l'implémentation la plus commune pour résoudre un jeu adversarielle avec des mouvements limités définis. Cet algorithme est basé sur le principe de construire un arbre avec les différents états possibles selon les actions de chaque joueur (AI et adversaire). L'algorithme veut maximiser les étages où le AI joue et minimiser les étages où l'adversaire joue. Dans ce cas-ci, l'AI déplace les voitures et l'adversaire place les roches. Dans le cadre du TP, l'algorithme n'explore pas jusqu'aux feuilles de l'arbre. Il s'arrête à une profondeur donnée.

La résolution du problème est faite par la classe **MiniMaxSearch** qui contient les champs suivants:

- `rushhour`: la classe Rushhour qui contient les informations sur la grille;
- `state`: la classe State avec l'état actuel de la grille;
- `search_depth`: la profondeur maximale d'exploration de l'algorithme;


In [21]:
import random 

class MiniMaxSearch:
    def __init__(self, rushHour, initial_state, search_depth):
        self.rushhour = rushHour
        self.state = initial_state
        self.search_depth = search_depth
        self.visited = set()
        self.alpha = None
        self.beta = None

    def minimax_1(self, current_depth, current_state):
        if current_depth == 0:
            score = current_state.score_state(self.rushhour)
            current_state.score = score
            return current_state

        pos_moves  = self.rushhour.possible_moves(current_state)
        best_score = float("inf")
        best_move = None
        for state in pos_moves:
            if state not in self.visited:
                child_state = self.minimax_1(current_depth -1, state)
                if child_state.score < best_score:
                    best_score = child_state.score
                    best_move = state
                    current_state.score = child_state.score

        if current_depth == self.search_depth:
            return best_move
        return current_state


    def minimax_2(self, current_depth, current_state, is_max): 
        if current_depth == 0:
            score = current_state.score_state(self.rushhour)
            current_state.score = score
            return current_state

        pos_moves  = None
        best_score = float("inf")
        if not is_max :
            pos_moves = self.rushhour.possible_moves(current_state)
        else:
            best_score = float("-inf")
            pos_moves = self.rushhour.possible_rock_moves(current_state)

        best_move = None
        for state in pos_moves:
            child_state = self.minimax_2(current_depth -1, state, not is_max)
            if not is_max: 
                if state not in self.visited:
                    if child_state.score < best_score:
                        best_score = child_state.score
                        best_move = state
                        current_state.score = child_state.score
            else:
                if child_state.score > best_score:
                    best_score = child_state.score
                    best_move = state
                    current_state.score = child_state.score


        if current_depth == self.search_depth:
            return best_move
        return current_state

    def minimax_pruning(self, current_depth, current_state, is_max, alpha, beta):
        #TODO
        if current_depth == 0:
            score = current_state.score_state(self.rushhour)
            current_state.score = score
            return current_state

        pos_moves  = None
        best_score = float("inf")
        if not is_max :
            pos_moves = self.rushhour.possible_moves(current_state)
        else:
            best_score = float("-inf")
            pos_moves = self.rushhour.possible_rock_moves(current_state)

        best_move = None
        for state in pos_moves:
            new_alpha = self.alpha if alpha == None else alpha
            new_beta = self.beta if beta == None else beta
            child_state = self.minimax_pruning(current_depth -1, state, not is_max, new_alpha, new_beta)                     
            if not is_max: 
                if state not in self.visited and child_state.score < best_score:
                    best_score = child_state.score
                    best_move = state
                    current_state.score = child_state.score
                if new_beta is not None and best_score < new_beta :
                    beta = best_score
                    break
            else:
                if child_state.score > best_score:
                    best_score = child_state.score
                    best_move = state
                    current_state.score = child_state.score

                if new_alpha is not None and best_score > new_alpha:
                    alpha = best_score
                    break


        if current_depth == 1:
            if is_max:
                self.alpha = best_score
            else:
                self.beta = best_score

        if current_depth == self.search_depth:
            return best_move
        return current_state

    def calculate_average(self, array):
        average = 0
        for value in array:
            average += value.score
        return average / len(array)

    def expectimax(self, current_depth, current_state, is_max):
        if current_depth == 0:
            score = current_state.score_state(self.rushhour)
            current_state.score = score
            return current_state

        best_move = current_state
        best_score = float("inf")
        if not is_max :
            pos_moves = self.rushhour.possible_moves(current_state)
        else:
            best_score = float("-inf")
            pos_moves = self.rushhour.possible_rock_moves(current_state)
        for state in pos_moves:
            child_state = self.expectimax(current_depth -1, state, not is_max)
            if not is_max: 
                if state not in self.visited:
                    if child_state.score < best_score:
                        best_score = child_state.score
                        best_move = state
                        current_state.score = child_state.score

        if is_max:
            average = self.calculate_average(pos_moves)
            for state in pos_moves:
                state.score = average
            return pos_moves[random.randint(0, len(pos_moves) - 1)]

        if current_depth == self.search_depth:
            return best_move
        return current_state

    def decide_best_move_1(self): # dispatcher
        if self.state.success() :
            return self.state
        else:
            return self.minimax_1(self.search_depth, self.state)

    def decide_best_move_2(self, is_max):
        if self.state.success() :
            return self.state
        else:
            return self.minimax_2(self.search_depth, self.state, is_max)

    def decide_best_move_pruning(self, is_max):
        # TODO
        if self.state.success() :
            return self.state
        else:
            return self.minimax_pruning(self.search_depth, self.state, is_max, None, None)


    def decide_best_move_expectimax(self, is_max):
        if self.state.success() :
            return self.state
        else:
            return self.expectimax(self.search_depth, self.state, is_max)

    def solve(self, state, is_singleplayer):
        if is_singleplayer:
            self.rushhour.print_pretty_grid(self.state)
            while(not self.state.success()):
                self.state = self.decide_best_move_1()
                self.visited.add(self.state)
                self.print_move(False, self.state)
               # self.rushhour.print_pretty_grid(self.state)
            print('fin apres ' + str(self.state.nb_moves) + " moves")
        else:
            self.rushhour.print_pretty_grid(self.state)
            counter = 0
            while(not self.state.success()):
                is_max = counter % 2 != 0
#                 self.state = self.decide_best_move_pruning(is_max)
#                 self.state = self.decide_best_move_2(is_max)
                self.state = self.decide_best_move_expectimax(is_max)
                self.visited.add(self.state)
                self.print_move(is_max, self.state)
                counter += 1
               # self.rushhour.print_pretty_grid(self.state)
            print('fin apres ' + str(self.state.nb_moves) + " moves")
        return self.state
   
    def print_move(self, is_max, state):
        #TODO
        if not is_max:
            if self.rushhour.horiz[state.c]:
                if state.d == 1:
                    print("Voiture ", self.rushhour.color[state.c], " vers la droite")
                else:
                    print("Voiture ", self.rushhour.color[state.c], " vers la gauche")
            else:
                if state.d == 1:
                    print("Voiture ", self.rushhour.color[state.c], " vers le haut")
                else:
                    print("Voiture ", self.rushhour.color[state.c], " vers le bas")
        else:
            print("Roche ", state.rock)
        

### 2.1 Implémentation simple
1. Implémenter la fonction *`score_state()`* de la classe **State**. Elle affecte la valeur de l'état à son paramètre score. L'état n'est pas nécessairement final. Utiliser l'heuristique qui vous semble la plus pertinente.

2. Implémenter la fonction  *`minimax_1()`*. Cette fonction contient la logique de l'algorithme minimax pour un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant.

3. Implémenter la fonction *`decide_best_move_1()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à un joueur.

4. Implémenter la fonction *`solve()`*. Cette fonction résout un problème de Rush Hour avec le nombre minimal de coups.

4. Implémenter la fonction *`print_move()`* pour imprimer le coup fait. Ex. "Voiture orange vers le haut" ou "Roche dans la case 2-1".

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

Utiliser la fonction _test_print_move()_ pour vérifier votre implémentation de _print_move()_. 

In [7]:
def test_print_move():
    rh = Rushhour([True], [2], [2], ["rouge"])
    s = State([0])
    s = s.put_rock((3,1)) # Roche dans la case 3-1
    s = s.move(0, 1) # Voiture rouge vers la droite

    algo = MiniMaxSearch(rh, s, 1)
    algo.print_move(True, s)
    algo.print_move(False, s)

test_print_move()

Roche  (3, 1)
Voiture  rouge  vers la droite


Pour vous aider dans l'implémentation de votre heuristique, 3 exemples de problèmes de Rush Hour sont fournis.

In [8]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture  bleu  vers le bas
Voiture  rouge  vers la droite
Voiture  orange  vers le bas
Voiture  jaune  vers la gauche
Voiture  vert  vers le haut
Voiture  vert  vers le haut
Voiture  vert  vers le haut
Voiture  rouge  vers la droite
Voiture  rouge  vers la droite
fin apres 9 moves
CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs


In [9]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Voiture  emeraude  vers la gauche
Voiture  lime  vers la gauche
Voiture  lime  vers la gauche
Voiture  lime  vers la gauche
Voiture  jaune  vers le haut
Voiture  jaune  vers le haut
Voiture  jaune  vers le haut
Voiture  vert  vers la droite
Voiture  vert  vers la droite
Voiture  vert  vers la droite
Voiture  vert  vers la droite
Voiture  mauve  vers le bas
Voiture  vert  vers la gauche
Voiture  vert  vers la gauche
Voiture  vert  vers la gauche
Voiture  orange  vers le bas
Voiture  emeraude  vers la gauche
Voiture  bleu  vers le haut
Voiture  bleu  vers le hau

In [10]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,1)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Voiture  rouge  vers la droite
Voiture  2  vers la gauche
Voiture  2  vers la gauche
Voiture  10  vers la gauche
Voiture  4  vers le bas
Voiture  4  vers le bas
Voiture  5  vers le haut
Voiture  5  vers le haut
Voiture  1  vers le haut
Voiture  2  vers la gauche
Voiture  3  vers le bas
Voiture  rouge  vers la droite
Voiture  rouge  vers la droite
Voiture  rouge  vers la droite
fin apres 14 moves
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


### 2.2 Implémentation adversarielle

1. Implémenter la fonction  *`minimax_2()`*. Cette fonction contient la logique de l'algorithme minimax pour deux joueurs et retourne le meilleur coup à prendre à partir de l'état courant.

2. Implémenter la fonction *`decide_best_move_2()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.

3. Modifier la fonction *`solve()`* pour qu'elle puisse résoudre des problèmes à deux joueurs.

4. Modifier la fonction *`score_state()`* de la classe **State** si elle ne tient pas déjà compte des roches.

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

 Le nombre donné de coups pour les 3 exemples suivants correspond à une partie sans adversaire. Le nombre de coups de plus dépend de l'implémentation de votre heuristique.

In [18]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture  vert  vers le haut
Roche  (0, 5)
Voiture  rouge  vers la droite
Roche  (1, 0)
Voiture  orange  vers le bas
Roche  (0, 5)
Voiture  vert  vers le haut
Roche  (1, 0)
Voiture  jaune  vers la gauche
Roche  (0, 1)
Voiture  vert  vers le haut
Roche  (1, 0)
Voiture  bleu  vers le haut
Roche  (4, 5)
Voiture  rouge  vers la droite
Roche  (0, 0)
Voiture  bleu  vers le bas
Roche  (1, 1)
Voiture  bleu  vers le bas
Roche  (0, 0)
Voiture  orange  vers le bas
Roche  (1, 2)
Voiture  orange  vers le bas
Roche  (0, 0)
Voiture  jaune  vers la gauche
Roche  (1, 2)
Voiture

In [19]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Voiture  vert  vers la droite
Roche  (0, 0)
Voiture  vert  vers la droite
Roche  (1, 1)
Voiture  vert  vers la droite
Roche  (0, 0)
Voiture  emeraude  vers la droite
Roche  (1, 1)
Voiture  vert  vers la gauche
Roche  (0, 0)
Voiture  lime  vers la gauche
Roche  (1, 1)
Voiture  vert  vers la droite
Roche  (0, 0)
Voiture  emeraude  vers la gauche
Roche  (1, 1)
Voiture  vert  vers la gauche
Roche  (0, 0)
Voiture  vert  vers la gauche
Roche  (1, 1)
Voiture  vert  vers la gauche
Roche  (0, 2)
Voiture  emeraude  vers la droite
Roche  (1, 1)
Voiture  lime  vers la gau

Voiture  emeraude  vers la droite
Roche  (1, 1)
Voiture  vert  vers la droite
Roche  (0, 2)
Voiture  vert  vers la droite
Roche  (1, 1)
Voiture  orange  vers le haut
Roche  (0, 2)
Voiture  vert  vers la gauche
Roche  (1, 1)
Voiture  vert  vers la gauche
Roche  (0, 4)
Voiture  mauve  vers le haut
Roche  (1, 1)
Voiture  vert  vers la droite
Roche  (0, 0)
Voiture  vert  vers la droite
Roche  (1, 1)
Voiture  emeraude  vers la gauche
Roche  (0, 0)
Voiture  vert  vers la gauche
Roche  (1, 1)
Voiture  mauve  vers le bas
Roche  (0, 2)
Voiture  vert  vers la droite
Roche  (1, 1)
Voiture  orange  vers le bas
Roche  (0, 2)
Voiture  vert  vers la gauche
Roche  (1, 1)
Voiture  emeraude  vers la gauche
Roche  (0, 2)
Voiture  vert  vers la droite
Roche  (1, 1)
Voiture  lime  vers la gauche
Roche  (0, 2)
Voiture  vert  vers la gauche
Roche  (1, 1)
Voiture  vert  vers la gauche
Roche  (0, 4)
Voiture  emeraude  vers la droite
Roche  (1, 1)
Voiture  vert  vers la droite
Roche  (0, 2)
Voiture  orange  ver

Voiture  vert  vers la gauche
Roche  (0, 1)
Voiture  mauve  vers le haut
Roche  (1, 2)
Voiture  vert  vers la gauche
Roche  (5, 5)
Voiture  vert  vers la gauche
Roche  (0, 2)
Voiture  emeraude  vers la droite
Roche  (1, 1)
Voiture  vert  vers la droite
Roche  (0, 0)
Voiture  vert  vers la droite
Roche  (1, 1)
Voiture  mauve  vers le bas
Roche  (0, 4)
Voiture  orange  vers le bas
Roche  (1, 1)
Voiture  emeraude  vers la gauche
Roche  (5, 5)
Voiture  vert  vers la droite
Roche  (0, 1)
Voiture  jaune  vers le haut
Roche  (1, 2)
Voiture  vert  vers la gauche
Roche  (0, 1)
Voiture  orange  vers le haut
Roche  (1, 2)
Voiture  vert  vers la droite
Roche  (0, 1)
Voiture  vert  vers la droite
Roche  (1, 2)
Voiture  mauve  vers le haut
Roche  (5, 1)


AttributeError: 'NoneType' object has no attribute 'c'

In [20]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Voiture  rouge  vers la droite
Roche  (0, 1)
Voiture  rouge  vers la gauche
Roche  (1, 2)
Voiture  2  vers la gauche
Roche  (0, 1)
Voiture  rouge  vers la droite
Roche  (1, 2)
Voiture  1  vers le haut
Roche  (0, 0)
Voiture  2  vers la gauche
Roche  (1, 1)
Voiture  2  vers la gauche
Roche  (0, 3)
Voiture  3  vers le haut
Roche  (1, 1)
Voiture  4  vers le bas
Roche  (0, 3)
Voiture  3  vers le haut
Roche  (1, 1)
Voiture  2  vers la droite
Roche  (0, 4)
Voiture  rouge  vers la droite
Roche  (1, 1)
Voiture  4  vers le bas
Roche  (0, 0)
Voiture  rouge  vers la gauch

Voiture  1  vers le bas
Roche  (1, 1)
Voiture  5  vers le haut
Roche  (0, 3)
Voiture  1  vers le haut
Roche  (1, 0)
Voiture  1  vers le haut
Roche  (0, 3)
Voiture  1  vers le haut
Roche  (1, 0)
Voiture  6  vers la droite
Roche  (0, 3)
Voiture  1  vers le bas
Roche  (1, 0)
Voiture  1  vers le bas
Roche  (0, 3)
Voiture  5  vers le bas
Roche  (1, 0)
Voiture  1  vers le haut
Roche  (0, 3)
Voiture  1  vers le haut
Roche  (1, 0)
Voiture  6  vers la droite
Roche  (0, 3)
Voiture  1  vers le bas
Roche  (1, 0)
Voiture  1  vers le bas
Roche  (0, 3)
Voiture  1  vers le bas
Roche  (3, 2)
Voiture  7  vers la gauche
Roche  (0, 3)
Voiture  1  vers le haut
Roche  (3, 2)
Voiture  1  vers le haut
Roche  (0, 3)
Voiture  6  vers la gauche
Roche  (1, 0)
Voiture  1  vers le bas
Roche  (0, 3)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  2  vers la droite
Roche  (0, 0)
Voiture  1  vers le haut
Roche  (1, 1)
Voiture  1  vers le haut
Roche  (0, 0)
Voiture  5  vers le haut
Roche  (1, 1)
Voiture  1  vers le bas


AttributeError: 'NoneType' object has no attribute 'c'

## 3. Élagage Alpha-Beta (15pts)
Pour accélérer la prise de décision, un élagage est effectué sur l'arbre. Cela permet de ne pas parcourir tout l'arbre. L'élagage alpha-beta est l'implémentation la plus fréquente pour résoudre ce genre de problème. En passant des valeurs d’alpha et de bêta récursivement, l'algorithme connaît la meilleure valeur trouvée à date. Les branches jugées non intéressantes, c'est-à-dire qu'elles ne peuvent pas produire un meilleur résultat que celui déjà trouvé, sont coupées dans le processus.

 
### 3.1 Implémentation

1. Implémenter la fonction *`minimax_pruning()`* pour qu'elle n'explore pas des branches inutiles lors d'une partie à deux joueurs.
2. Implémenter la fonction *`decide_best_move_pruning()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Vérifier que cette nouvelle méthode visite bel et bien moins de noeuds. **Montrer cette information dans un tableau**.

In [14]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture  vert  vers le haut
Roche  (0, 5)
Voiture  rouge  vers la droite
Roche  (1, 0)
Voiture  vert  vers le haut
Roche  (0, 5)
Voiture  rouge  vers la gauche
Roche  (1, 0)
Voiture  bleu  vers le haut
Roche  (4, 5)
Voiture  rouge  vers la droite
Roche  (0, 0)
Voiture  orange  vers le bas
Roche  (4, 5)
Voiture  vert  vers le bas
Roche  (0, 0)
Voiture  bleu  vers le bas
Roche  (1, 1)
Voiture  vert  vers le haut
Roche  (0, 5)
Voiture  orange  vers le bas
Roche  (1, 0)
Voiture  jaune  vers la gauche
Roche  (0, 1)
Voiture  vert  vers le haut
Roche  (1, 0)
Voiture 

In [15]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Voiture  vert  vers la droite
Roche  (0, 0)
Voiture  vert  vers la droite
Roche  (1, 1)
Voiture  vert  vers la droite
Roche  (0, 0)
Voiture  emeraude  vers la droite
Roche  (1, 1)
Voiture  vert  vers la gauche
Roche  (0, 0)
Voiture  lime  vers la gauche
Roche  (1, 1)
Voiture  vert  vers la droite
Roche  (0, 0)
Voiture  emeraude  vers la gauche
Roche  (1, 1)
Voiture  vert  vers la gauche
Roche  (0, 0)
Voiture  vert  vers la gauche
Roche  (1, 1)
Voiture  vert  vers la gauche
Roche  (0, 2)
Voiture  emeraude  vers la droite
Roche  (1, 1)
Voiture  lime  vers la gau

Roche  (0, 1)
Voiture  jaune  vers le haut
Roche  (1, 2)
Voiture  vert  vers la gauche
Roche  (0, 1)
Voiture  vert  vers la gauche
Roche  (1, 2)
Voiture  bleu  vers le haut
Roche  (5, 3)
Voiture  vert  vers la droite
Roche  (0, 1)
Voiture  emeraude  vers la droite
Roche  (1, 2)
Voiture  orange  vers le haut
Roche  (0, 1)
Voiture  vert  vers la gauche
Roche  (1, 2)
Voiture  vert  vers la gauche
Roche  (0, 3)
Voiture  mauve  vers le haut
Roche  (1, 1)
Voiture  vert  vers la droite
Roche  (0, 0)
Voiture  emeraude  vers la droite
Roche  (1, 1)
Voiture  vert  vers la droite
Roche  (0, 0)
Voiture  jaune  vers le bas
Roche  (1, 1)
Voiture  vert  vers la droite
Roche  (0, 0)
Voiture  emeraude  vers la droite
Roche  (1, 1)
Voiture  mauve  vers le bas
Roche  (0, 2)
Voiture  vert  vers la gauche
Roche  (1, 1)
Voiture  vert  vers la gauche
Roche  (0, 4)
Voiture  mauve  vers le haut
Roche  (1, 1)
Voiture  vert  vers la gauche
Roche  (5, 2)
Voiture  vert  vers la gauche
Roche  (0, 3)
Voiture  emerau

AttributeError: 'NoneType' object has no attribute 'c'

In [16]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Voiture  rouge  vers la droite
Roche  (0, 1)
Voiture  rouge  vers la gauche
Roche  (1, 2)
Voiture  2  vers la gauche
Roche  (0, 1)
Voiture  rouge  vers la droite
Roche  (1, 2)
Voiture  1  vers le haut
Roche  (0, 0)
Voiture  2  vers la gauche
Roche  (1, 1)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  rouge  vers la gauche
Roche  (4, 3)
Voiture  3  vers le haut
Roche  (0, 4)
Voiture  3  vers le haut
Roche  (1, 1)
Voiture  rouge  vers la droite
Roche  (0, 4)
Voiture  1  vers le haut
Roche  (1, 1)
Voiture  2  vers la droite
Roche  (0, 0)
Voiture  rouge  vers la 

Voiture  8  vers le bas
Roche  (0, 3)
Voiture  1  vers le bas
Roche  (1, 0)
Voiture  2  vers la droite
Roche  (0, 4)
Voiture  1  vers le haut
Roche  (1, 0)
Voiture  9  vers la droite
Roche  (0, 4)
Voiture  1  vers le bas
Roche  (1, 0)
Voiture  2  vers la gauche
Roche  (0, 3)
Voiture  1  vers le haut
Roche  (1, 0)
Voiture  3  vers le bas
Roche  (0, 4)
Voiture  1  vers le bas
Roche  (1, 0)
Voiture  7  vers la droite
Roche  (0, 4)
Voiture  1  vers le haut
Roche  (1, 0)
Voiture  3  vers le haut
Roche  (0, 3)
Voiture  1  vers le bas
Roche  (1, 0)
Voiture  2  vers la droite
Roche  (0, 4)
Voiture  1  vers le haut
Roche  (1, 0)
Voiture  10  vers la gauche
Roche  (0, 4)
Voiture  1  vers le bas
Roche  (1, 0)
Voiture  2  vers la gauche
Roche  (0, 3)
Voiture  1  vers le haut
Roche  (1, 0)
Voiture  3  vers le bas
Roche  (0, 4)
Voiture  1  vers le bas
Roche  (1, 0)
Voiture  7  vers la gauche
Roche  (0, 4)
Voiture  1  vers le haut
Roche  (1, 0)
Voiture  3  vers le haut
Roche  (0, 3)
Voiture  1  vers 

## 4. Expectimax (15 pts)
L'expectimax est une variante de la recherche minimax. Elle est utilisée lorsqu'il y a une composante de hasard lors de la prise de décision. Depuis le début du TP, il a été supposé que l'adversaire prenait toujours le coup optimal. Maintenant, l'adversaire va faire un coup aléatoirement. Pour résoudre ce cas-là, la recherche expectimax sera utilisée.

### 4.1 Implémentation 

1. Implémenter la fonction *`minimax_expectimax()`* pour qu'elle implémente l'algorithme de recherche. Considérer une probabilité égale pour tous les coups.
2. Implémenter la fonction *`decide_best_move_expectimax()`*. Cette fonction trouve et exécute le meilleur coup pour une partie avec un adversaire imprévisible.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Modifier les valeurs de probabilités associées aux coups pour donner une vision optimiste et une vision pessimiste à l'AI. Comparer la performance du AI entre les 3 visions (aléatoire, optimiste, pessimiste). Attention l'adversaire continue à prendre ses décisions au hasard. **Montrer ces informations dans un tableau et les analyser**. 

In [22]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture  vert  vers le haut
Roche  (3, 3)
Voiture  bleu  vers le haut
Roche  (4, 5)
Voiture  rouge  vers la droite
Roche  (0, 0)
Voiture  bleu  vers le bas
Roche  (4, 3)
Voiture  orange  vers le bas
Roche  (3, 0)
Voiture  bleu  vers le haut
Roche  (0, 3)
Voiture  vert  vers le bas
Roche  (4, 4)
Voiture  orange  vers le haut
Roche  (0, 0)
Voiture  bleu  vers le bas
Roche  (1, 2)
Voiture  orange  vers le bas
Roche  (4, 4)
Voiture  orange  vers le bas
Roche  (0, 2)
Voiture  vert  vers le haut
Roche  (4, 4)
Voiture  bleu  vers le haut
Roche  (0, 0)
Voiture  vert  

In [23]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Voiture  vert  vers la droite
Roche  (1, 2)
Voiture  vert  vers la droite
Roche  (3, 1)
Voiture  vert  vers la droite
Roche  (0, 2)
Voiture  mauve  vers le bas
Roche  (5, 5)
Voiture  vert  vers la gauche
Roche  (3, 2)
Voiture  vert  vers la gauche
Roche  (0, 3)
Voiture  orange  vers le bas
Roche  (5, 1)
Voiture  vert  vers la droite
Roche  (4, 3)
Voiture  vert  vers la droite
Roche  (5, 5)
Voiture  emeraude  vers la gauche
Roche  (0, 2)
Voiture  orange  vers le haut
Roche  (4, 1)
Voiture  vert  vers la gauche
Roche  (3, 2)
Voiture  mauve  vers le haut
Roche  (

Voiture  vert  vers la droite
Roche  (3, 1)
Voiture  bleu  vers le bas
Roche  (1, 4)
Voiture  vert  vers la droite
Roche  (0, 1)
Voiture  vert  vers la droite
Roche  (4, 3)
Voiture  emeraude  vers la gauche
Roche  (5, 5)
Voiture  orange  vers le haut
Roche  (4, 4)
Voiture  bleu  vers le haut
Roche  (1, 5)
Voiture  jaune  vers le haut
Roche  (3, 4)
Voiture  orange  vers le bas
Roche  (0, 1)
Voiture  emeraude  vers la gauche
Roche  (5, 3)
Voiture  bleu  vers le bas
Roche  (0, 2)
Voiture  vert  vers la gauche
Roche  (3, 1)
Voiture  vert  vers la gauche
Roche  (4, 3)
Voiture  jaune  vers le bas
Roche  (5, 5)
Voiture  vert  vers la gauche
Roche  (3, 1)
Voiture  bleu  vers le haut
Roche  (0, 4)
Voiture  bleu  vers le haut
Roche  (1, 3)
Voiture  vert  vers la droite
Roche  (0, 1)
Voiture  rouge  vers la droite
Roche  (1, 4)
Voiture  vert  vers la droite
Roche  (0, 2)
Voiture  vert  vers la droite
Roche  (1, 4)
Voiture  rouge  vers la droite
Roche  (3, 2)
Voiture  vert  vers la gauche
Roche  (

In [24]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
Voiture  3  vers le haut
Roche  (1, 1)
Voiture  4  vers le bas
Roche  (0, 2)
Voiture  rouge  vers la droite
Roche  (4, 1)
Voiture  1  vers le haut
Roche  (0, 2)
Voiture  4  vers le haut
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (0, 2)
Voiture  9  vers la droite
Roche  (4, 0)
Voiture  rouge  vers la gauche
Roche  (1, 2)
Voiture  2  vers la gauche
Roche  (4, 3)
Voiture  rouge  vers la droite
Roche  (1, 2)
Voiture  1  vers le haut
Roche  (0, 5)
Voiture  2  vers la gauche
Roche  (1, 4)
Voiture  2  vers la gauche
Roche  (5, 3)
Voiture  3  vers le bas
Roche  (1, 

Voiture  1  vers le haut
Roche  (4, 5)
Voiture  1  vers le haut
Roche  (5, 3)
Voiture  rouge  vers la gauche
Roche  (1, 1)
Voiture  5  vers le haut
Roche  (5, 3)
Voiture  rouge  vers la droite
Roche  (1, 2)
Voiture  1  vers le bas
Roche  (0, 3)
Voiture  1  vers le bas
Roche  (1, 5)
Voiture  2  vers la droite
Roche  (5, 3)
Voiture  1  vers le haut
Roche  (1, 5)
Voiture  6  vers la gauche
Roche  (4, 0)
Voiture  4  vers le haut
Roche  (1, 2)
Voiture  2  vers la gauche
Roche  (5, 3)
Voiture  1  vers le haut
Roche  (1, 0)
Voiture  5  vers le bas
Roche  (5, 3)
Voiture  1  vers le bas
Roche  (4, 1)
Voiture  1  vers le bas
Roche  (5, 3)
Voiture  4  vers le bas
Roche  (0, 4)
Voiture  2  vers la droite
Roche  (5, 3)
Voiture  5  vers le bas
Roche  (1, 2)
Voiture  2  vers la gauche
Roche  (3, 4)
Voiture  1  vers le haut
Roche  (1, 0)
Voiture  1  vers le haut
Roche  (3, 5)
Voiture  rouge  vers la gauche
Roche  (1, 2)
Voiture  6  vers la droite
Roche  (3, 1)
Voiture  rouge  vers la droite
Roche  (4,

Roche  (0, 4)
Voiture  1  vers le haut
Roche  (5, 3)
Voiture  6  vers la droite
Roche  (4, 1)
Voiture  rouge  vers la gauche
Roche  (5, 3)
Voiture  1  vers le bas
Roche  (4, 0)
Voiture  rouge  vers la droite
Roche  (3, 5)
Voiture  10  vers la droite
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  6  vers la gauche
Roche  (5, 3)
Voiture  10  vers la gauche
Roche  (4, 0)
Voiture  6  vers la gauche
Roche  (1, 1)
Voiture  rouge  vers la gauche
Roche  (0, 4)
Voiture  3  vers le haut
Roche  (4, 0)
Voiture  5  vers le haut
Roche  (0, 5)
Voiture  2  vers la droite
Roche  (3, 3)
Voiture  9  vers la gauche
Roche  (5, 5)
Voiture  2  vers la gauche
Roche  (4, 0)
Voiture  5  vers le haut
Roche  (3, 3)
Voiture  2  vers la droite
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  rouge  vers la gauche
Roche  (5, 5)
Voiture  2  vers la droite
Roche  (4, 1)
Voiture  2  vers la droite
Roche  (1, 2)
Voiture  rouge  vers la droite
Roche  (4, 0)
Voiture  1  vers le haut
Roche  (1, 2)

Roche  (3, 0)
Voiture  2  vers la droite
Roche  (1, 3)
Voiture  1  vers le haut
Roche  (4, 1)
Voiture  1  vers le haut
Roche  (1, 3)
Voiture  2  vers la gauche
Roche  (4, 1)
Voiture  1  vers le bas
Roche  (0, 3)
Voiture  5  vers le bas
Roche  (4, 1)
Voiture  1  vers le haut
Roche  (1, 3)
Voiture  2  vers la droite
Roche  (4, 1)
Voiture  1  vers le bas
Roche  (0, 0)
Voiture  1  vers le bas
Roche  (1, 3)
Voiture  2  vers la gauche
Roche  (4, 0)
Voiture  6  vers la gauche
Roche  (3, 4)
Voiture  rouge  vers la gauche
Roche  (0, 3)
Voiture  1  vers le haut
Roche  (4, 0)
Voiture  2  vers la droite
Roche  (3, 4)
Voiture  1  vers le haut
Roche  (1, 1)
Voiture  2  vers la gauche
Roche  (3, 4)
Voiture  rouge  vers la droite
Roche  (1, 2)
Voiture  2  vers la droite
Roche  (0, 0)
Voiture  1  vers le bas
Roche  (3, 5)
Voiture  1  vers le bas
Roche  (0, 0)
Voiture  rouge  vers la gauche
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (4, 0)
Voiture  rouge  vers la gauche
Roche  (1, 2)
Voiture  4  vers 

Voiture  1  vers le bas
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (4, 1)
Voiture  1  vers le bas
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (4, 1)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  vers le bas
Roche  (4, 1)
Voiture  1  

...

Voiture  1  vers le bas
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  vers le bas
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  vers le bas
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  

Voiture  1  vers le bas
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (4, 1)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (4, 1)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (4, 1)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (4, 1)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  

Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (4, 0

Voiture  1  vers le bas
Roche  (1, 1)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (4, 1)
Voiture  1  vers le bas
Roche  (1, 3)
Voiture  1  vers le bas
Roche  (4, 1)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  vers le bas
Roche  (4, 2)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (3, 0)
Voiture  1  vers le bas
Roche  (0, 5)
Voiture  1  vers le bas
Roche  (4, 1)
Voiture  1  vers le bas
Roche  (5, 0)
Voiture  1  vers le bas
Roche  (1, 1)
Voiture  1  vers le bas
Roche  (4, 0)
Voiture  1  vers le bas
Roche  (3, 1)
Voiture  1  

KeyboardInterrupt: 

## 5. Comparaison (10pts)
Comparez la performance la recherche minimax avec la recherche expectimax lorsque l'adversaire est optimale et lorsque l'adversaire est aléatoire. Pour un adversaire aléatoire, prenez une vision neutre. Roulez plusieurs fois les algorithmes (vous pouvez aussi reprendre les tests du TP1) et notez si c'est une victoire ou une défaire pour l'AI ainsi que le nombre de coups moyen, minimal et maximal. Basez-vous sur le nombre de coups optimaux pour résoudre les problèmes pour déterminer un seuil de victoire et de défaite pour l'AI. Précisez votre seuil. **Mettez le tout dans un tableau et analysez vos résultats**. 

### Comparaison : minimax vs expectimax
À noter que nous avons des erreur pour minimax 16 coups et expectimax 14, donc on va comparer uniquement les deux avec 9 coups.
Nous avons que le minimax avec 9 coups est meilleur que expectimax. Cela est du fait que notre fonction score n'est pas parfaite donc une approche aleatoire pour la roche peut faire en sorte de bloquer des états important (dependance de bloquage) Nous avons été surpis de ce résultat, car nous pensions que l'approche aléatoire est moins efficace pour la roche afin de bloquer les mouvements important du joueur.

In [29]:
# !pip3 install tabulate
from IPython.display import HTML, display
import tabulate
table = [["","9 coups","16 coups","14 coups"],
         ["MiniMax",16,"ERROR",374],
         ["ExpectiMax Aleatoire",61,228,"ERROR"]]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

,9 coups,16 coups,14 coups
MiniMax,16,ERROR,374
ExpectiMax Aleatoire,61,228,ERROR
